In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from pycocotools.coco import COCO
from pycocotools.cbas import CBAS
import cbas_construction_utils as ccu
import sklearn as sk

#### Load COCO

In [28]:
# We'll create CBAS-80 out of the COCO "train2017" image set
originSet = 'train2017'

# initialize COCO api for instance annotations and category info
coco=COCO('../annotations/instances_{}.json'.format(originSet))

# This doesn't work because I don't have train2017, but i think i should base it off of cbas_80
# so I'll load this in later when i can get the info from Mike 

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../annotations/instances_train2017.json'

### Load cbas_34:

In [29]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])

# trainset = base classes 
base_trainset = torchvision.datasets.ImageFolder(root='../images/cbas34_train', transform=transform)


In [30]:
print(base_trainset.__dict__.keys())
print(base_trainset.imgs[3])
print("Base classes: ", base_trainset.classes)

dict_keys(['root', 'imgs', 'classes', 'class_to_idx', 'transform', 'target_transform', 'loader'])
('../images/cbas34_train/airplane/1363089.jpg', 0)
Base classes:  ['airplane', 'backpack', 'banana', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'car', 'carrot', 'chair', 'clock', 'cow', 'cup', 'donut', 'fork', 'handbag', 'horse', 'kite', 'knife', 'person', 'pottedplant', 'sheep', 'sink', 'skateboard', 'spoon', 'surfboard', 'tennisracket', 'trafficlight', 'truck', 'umbrella', 'vase']


#### Find the holdout categories: 

In [26]:
all_categories = COCO.loadCats(COCO.getCatIds())
nms=[cat['name'] for cat in all_categories]
print('COCO categories: \n{}\n'.format(' '.join(nms)))


TypeError: getCatIds() missing 1 required positional argument: 'self'

#### Train network that learns to predict images from base_set: 

In [ ]:
# Load the pretrained model
model = models.resnet18(pretrained=True)